In [43]:
d1 = 'I thought, I THOUGHT of Thinking of thanking you for the gift.'
d2 = 'She was thinking of going to go, and get you a GIFT! but She don\\'' know to buying it'

# Le script ci-dessous peut vous aider à télécharger ces corpus. 
Si vous avez déjà téléchargé, l'exécuter vous avertira qu'il est à jour

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/simplon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/simplon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# préparons l'environnement avec des packages et des données:

In [44]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

#creer un dataframe
X_train = pd.DataFrame({'text': [d1, d2]})

In [45]:
X_train

,text
0,"I thought, I THOUGHT of Thinking of thanking y..."
1,"She was thinking of going to go, and get you a..."


In [7]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords

In [14]:
d1_norm = preprocess_text(d1)
d2_norm = preprocess_text(d2)
print('d1 = ',d1_norm)
print('d2 = ',d2_norm)

d1 =  ['think', 'think', 'think', 'thank', 'gift']
d2 =  ['think', 'go', 'go', 'get', 'gift']


# prétraitez le corpus

In [46]:
# Create an instance of TfidfVectorizer
vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# Fit to the data and transform to feature matrix
X_train = vectoriser.fit_transform(X_train['text'])
X_train

<2x7 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [47]:

# Convert sparse matrix to dataframe
X_train = pd.DataFrame.sparse.from_spmatrix(X_train)
X_train

,0,1,2,3,4,5,6
0,0.000000,0.000000,0.288972,0.000000,0.000000,0.40614,0.866917
1,0.353278,0.353278,0.251360,0.706556,0.353278,0.00000,0.251360


In [48]:
# Save mapping on which index refers to which words
col_map = {v:k for k, v in vectoriser.vocabulary_.items()}
col_map

{6: 'think', 5: 'thank', 2: 'gift', 3: 'go', 1: 'get', 4: 'know', 0: 'buy'}

In [49]:
# Rename each column using the mapping
for col in X_train.columns:
    X_train.rename(columns={col: col_map[col]}, inplace=True)


In [50]:
print("Matrice TF-IDF")
print(X_train)

Matrice TF-IDF
        buy       get      gift        go      know    thank     think
0  0.000000  0.000000  0.288972  0.000000  0.000000  0.40614  0.866917
1  0.353278  0.353278  0.251360  0.706556  0.353278  0.00000  0.251360


# TEST avec les documents avec ou sans termes invisibles

In [51]:
# 2 documents de tests
d3 = "He thinks, he will GO!"
d4 = "They don’t Know what to buy!"
#creer DF
X_test = pd.DataFrame({'text': [d3, d4]})
#Transform en matrice de feature
X_test = vectoriser.transform(X_test['text'])
X_test

<2x7 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [52]:
#convertir sparse matrix en DF
X_test = pd.DataFrame.sparse.from_spmatrix(X_test)
X_test

,0,1,2,3,4,5,6
0,0.000000,0.0,0.0,0.814802,0.000000,0.0,0.579739
1,0.707107,0.0,0.0,0.000000,0.707107,0.0,0.000000


In [53]:
#ajoute des noms du col
for col in X_test.columns:
    X_test.rename(columns={col: col_map[col]}, inplace=True)
X_test

,buy,get,gift,go,know,thank,think
0,0.000000,0.0,0.0,0.814802,0.000000,0.0,0.579739
1,0.707107,0.0,0.0,0.000000,0.707107,0.0,0.000000


In [42]:
#d3 = preprocess_text(d3)
d4 = preprocess_text(d4)
print(d3)
print(d4)

['think', 'go']
['know', 'buy']
